In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.model_selection import GroupKFold,TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_set.csv')

/tmp/ipykernel_61462/1480996787.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [ ]:
for df in train,test:
    df['kickoff_time'] = pd.to_datetime(df['kickoff_time'])
    

In [ ]:
train['name'].nunique()

1017

In [ ]:
to_shift_features = ('goals_scored','goals_conceded','assists','bonus'
                     ,'saves','own_goals','penalties_missed','penalties_saved',
                     'red_cards','clean_sheets','minutes','yellow_cards')


normal_features = (
    'kickoff_time','opponent_team','opp_team_name','round','value','was_home','name','position'
)

In [ ]:
train.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'season_x', 'name', 'position', 'team_x',
       'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [ ]:
all_feats = to_shift_features+normal_features

In [ ]:
train

,Unnamed: 0.1,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,454,10,0,0,0.0,0.0,2016-08-15 19:00:00+00:00,0,4,Chelsea,0,0,0,0,1,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,142,3,0,0,0.9,8.2,2016-08-13 14:00:00+00:00,15,17,Spurs,0,0,0,0,1,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,16,8,3,0,3.0,2.2,2016-08-14 15:00:00+00:00,60,9,Liverpool,0,0,0,0,1,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,482,7,0,0,0.0,0.0,2016-08-13 14:00:00+00:00,0,13,Southampton,0,0,0,0,1,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,286,6,1,0,0.3,2.0,2016-08-13 14:00:00+00:00,69,14,Stoke,0,0,0,0,1,0,2723,1.0,1.0,0.0,1,0,0,0,45,True,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101483,101483,101483,2021-22,Wilfred Ndidi,MID,Leicester,0,0,0,0,0.0,216,377,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,0,16,Southampton,0,0,0,0,38,0,70366,1.0,4.0,0.0,0,-202,22,224,48,True,0,38
101484,101484,101484,2021-22,Matt Ritchie,DEF,Newcastle,0,0,3,0,0.0,292,374,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,1,5,Burnley,0,0,0,0,38,0,15999,2.0,1.0,0.0,1,143,396,253,49,False,0,38
101485,101485,101485,2021-22,Nathan Redmond,MID,Southampton,0,0,5,0,0.0,336,377,3,0,0.0,0.0,2022-05-22 15:00:00+00:00,82,9,Leicester,0,0,0,0,38,0,13856,1.0,4.0,0.0,2,455,683,228,59,False,0,38
101486,101486,101486,2021-22,Mathew Ryan,GK,Brighton,0,0,0,0,0.0,65,373,0,0,0.0,0.0,2022-05-22 15:00:00+00:00,0,19,West Ham,0,0,0,0,38,0,587,1.0,3.0,0.0,0,-2,0,2,45,True,0,38


In [ ]:

def apply_feature_engineering(df):
    df = df.copy()
    cv = GroupKFold(n_splits=df['name'].nunique())
    frames = []
    for _,data in cv.split(df,groups=df['name']):
        data = df.iloc[data]
        data.sort_values('kickoff_time')
        for feat in to_shift_features:
            data[feat] = data[feat].shift(1)

        frames.append(data)
    data = pd.concat(frames)
        
    return data

train = apply_feature_engineering(train)
test = apply_feature_engineering(test)


/tmp/ipykernel_61462/896811144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].shift(1)
/tmp/ipykernel_61462/896811144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].shift(1)


In [ ]:
target = train['total_points']

In [ ]:
train

,Unnamed: 0.1,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
24,24,24,2016-17,Ben Davies,DEF,NaN,NaN,NaN,0,NaN,0.0,386,3,NaN,NaN,0.0,0.0,2016-08-13 14:00:00+00:00,NaN,6,Everton,NaN,NaN,NaN,NaN,1,NaN,9018,1.0,1.0,0.0,0,0,0,0,50,False,NaN,1
223,223,223,2016-17,Ben Davies,DEF,NaN,0.0,0.0,0,0.0,0.0,386,98,0.0,0.0,0.0,0.0,2016-10-29 14:00:00+00:00,0.0,8,Leicester,0.0,0.0,0.0,0.0,10,0.0,11191,1.0,1.0,0.0,0,-721,167,888,49,True,0.0,10
441,441,441,2016-17,Ben Davies,DEF,NaN,0.0,0.0,0,0.0,0.0,386,101,0.0,0.0,0.0,0.0,2016-11-06 12:00:00+00:00,0.0,1,Arsenal,0.0,0.0,0.0,0.0,11,0.0,10303,1.0,1.0,0.0,0,-998,124,1122,48,False,0.0,11
659,659,659,2016-17,Ben Davies,DEF,NaN,0.0,0.0,0,0.0,0.0,386,118,0.0,0.0,0.0,0.0,2016-11-19 17:30:00+00:00,0.0,20,West Ham,0.0,0.0,0.0,0.0,12,0.0,9163,2.0,3.0,0.0,0,-1168,26,1194,48,True,0.0,12
877,877,877,2016-17,Ben Davies,DEF,NaN,0.0,0.0,0,0.0,0.0,386,123,0.0,0.0,0.0,0.0,2016-11-26 17:30:00+00:00,0.0,4,Chelsea,0.0,0.0,0.0,0.0,13,0.0,8724,1.0,2.0,0.0,0,-444,12,456,48,False,0.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75859,75859,75859,2020-21,William Osula,FWD,Sheffield Utd,NaN,NaN,0,NaN,0.0,711,376,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,4,Burnley,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,True,NaN,38
75986,75986,75986,2020-21,Nile John,MID,Spurs,NaN,NaN,0,NaN,0.0,710,373,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,9,Leicester,NaN,NaN,NaN,NaN,38,NaN,0,4.0,2.0,0.0,0,0,0,0,45,False,NaN,38
101080,101080,101080,2021-22,Charlie Savage,MID,Man Utd,NaN,NaN,0,NaN,0.0,737,376,NaN,NaN,0.0,0.0,2022-05-22 15:00:00+00:00,NaN,7,Crystal Palace,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,False,NaN,38
75740,75740,75740,2020-21,Sylvester Jasper,MID,Fulham,NaN,NaN,0,NaN,0.0,707,371,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,14,Newcastle,NaN,NaN,NaN,NaN,38,NaN,0,2.0,0.0,0.0,0,0,0,0,45,True,NaN,38


In [ ]:
le = LabelEncoder()

In [ ]:
categorical = ['position','was_home']

In [ ]:
for category in categorical:
    le.fit(train[category])
    for df in train,test:
        df[category] = le.transform(df[category])

In [ ]:
train

,Unnamed: 0.1,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,opp_team_name,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
24,24,24,2016-17,Ben Davies,0,NaN,NaN,NaN,0,NaN,0.0,386,3,NaN,NaN,0.0,0.0,2016-08-13 14:00:00+00:00,NaN,6,Everton,NaN,NaN,NaN,NaN,1,NaN,9018,1.0,1.0,0.0,0,0,0,0,50,0,NaN,1
223,223,223,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,98,0.0,0.0,0.0,0.0,2016-10-29 14:00:00+00:00,0.0,8,Leicester,0.0,0.0,0.0,0.0,10,0.0,11191,1.0,1.0,0.0,0,-721,167,888,49,1,0.0,10
441,441,441,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,101,0.0,0.0,0.0,0.0,2016-11-06 12:00:00+00:00,0.0,1,Arsenal,0.0,0.0,0.0,0.0,11,0.0,10303,1.0,1.0,0.0,0,-998,124,1122,48,0,0.0,11
659,659,659,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,118,0.0,0.0,0.0,0.0,2016-11-19 17:30:00+00:00,0.0,20,West Ham,0.0,0.0,0.0,0.0,12,0.0,9163,2.0,3.0,0.0,0,-1168,26,1194,48,1,0.0,12
877,877,877,2016-17,Ben Davies,0,NaN,0.0,0.0,0,0.0,0.0,386,123,0.0,0.0,0.0,0.0,2016-11-26 17:30:00+00:00,0.0,4,Chelsea,0.0,0.0,0.0,0.0,13,0.0,8724,1.0,2.0,0.0,0,-444,12,456,48,0,0.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75859,75859,75859,2020-21,William Osula,1,Sheffield Utd,NaN,NaN,0,NaN,0.0,711,376,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,4,Burnley,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,1,NaN,38
75986,75986,75986,2020-21,Nile John,4,Spurs,NaN,NaN,0,NaN,0.0,710,373,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,9,Leicester,NaN,NaN,NaN,NaN,38,NaN,0,4.0,2.0,0.0,0,0,0,0,45,0,NaN,38
101080,101080,101080,2021-22,Charlie Savage,4,Man Utd,NaN,NaN,0,NaN,0.0,737,376,NaN,NaN,0.0,0.0,2022-05-22 15:00:00+00:00,NaN,7,Crystal Palace,NaN,NaN,NaN,NaN,38,NaN,0,0.0,1.0,0.0,0,0,0,0,45,0,NaN,38
75740,75740,75740,2020-21,Sylvester Jasper,4,Fulham,NaN,NaN,0,NaN,0.0,707,371,NaN,NaN,0.0,0.0,2021-05-23 15:00:00+00:00,NaN,14,Newcastle,NaN,NaN,NaN,NaN,38,NaN,0,2.0,0.0,0.0,0,0,0,0,45,1,NaN,38


In [ ]:
train['Unnamed: 0']

24            24
223          223
441          441
659          659
877          877
           ...  
75859      75859
75986      75986
101080    101080
75740      75740
75631      75631
Name: Unnamed: 0, Length: 101488, dtype: int64

In [ ]:
name = test['name']

In [ ]:
for df in train,test:
    df.set_index('kickoff_time',inplace=True)
    df.drop(columns=['Unnamed: 0','Unnamed: 0.1','name','season_x','opp_team_name',
                     'team_x','ict_index','influence','element','threat','transfers_in','transfers_out','value','creativity',
                     'team_a_score','team_h_score','transfers_balance'],inplace=True)

In [ ]:
train

,position,assists,bonus,bps,clean_sheets,fixture,goals_conceded,goals_scored,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,total_points,was_home,yellow_cards,GW
kickoff_time,,,,,,,,,,,,,,,,,,,,,
2016-08-13 14:00:00+00:00,0,NaN,NaN,0,NaN,3,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,1,NaN,9018,0,0,NaN,1
2016-10-29 14:00:00+00:00,0,0.0,0.0,0,0.0,98,0.0,0.0,0.0,8,0.0,0.0,0.0,0.0,10,0.0,11191,0,1,0.0,10
2016-11-06 12:00:00+00:00,0,0.0,0.0,0,0.0,101,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,11,0.0,10303,0,0,0.0,11
2016-11-19 17:30:00+00:00,0,0.0,0.0,0,0.0,118,0.0,0.0,0.0,20,0.0,0.0,0.0,0.0,12,0.0,9163,0,1,0.0,12
2016-11-26 17:30:00+00:00,0,0.0,0.0,0,0.0,123,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,13,0.0,8724,0,0,0.0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-23 15:00:00+00:00,1,NaN,NaN,0,NaN,376,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,38,NaN,0,0,1,NaN,38
2021-05-23 15:00:00+00:00,4,NaN,NaN,0,NaN,373,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,38,NaN,0,0,0,NaN,38
2022-05-22 15:00:00+00:00,4,NaN,NaN,0,NaN,376,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,38,NaN,0,0,0,NaN,38


In [ ]:
from lightgbm import LGBMRegressor

In [ ]:
model = LGBMRegressor(random_state=42)

In [ ]:
train['total_points']

kickoff_time
2016-08-13 14:00:00+00:00    0
2016-10-29 14:00:00+00:00    0
2016-11-06 12:00:00+00:00    0
2016-11-19 17:30:00+00:00    0
2016-11-26 17:30:00+00:00    0
                            ..
2021-05-23 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    0
2022-05-22 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    0
2021-05-23 15:00:00+00:00    1
Name: total_points, Length: 101488, dtype: int64

In [ ]:
cv = TimeSeriesSplit(n_splits=10)


metric = [] 
for train_v,test_v in cv.split(train):
    train_set = train.iloc[train_v]
    validation = train.iloc[test_v]
    model.fit(train_set.drop(columns='total_points'),train_set['total_points'])
    predictions = model.predict(validation.drop(columns='total_points'))
    metric.append(mean_squared_error(validation['total_points'],predictions))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 826
[LightGBM] [Info] Number of data points in the train set: 9228, number of used features: 20
[LightGBM] [Info] Start training from score 2.126463
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 837
[LightGBM] [Info] Number of data points in the train set: 18454, number of used features: 20
[LightGBM] [Info] Start training from score 2.161266
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [ ]:
metric

[1.0849452909102781,
 0.8903206439524439,
 0.7634775633767649,
 0.6986939814189524,
 0.6648425114781534,
 0.8184923404747616,
 0.5461850356747887,
 0.522623178278505,
 0.44921569561721536,
 0.3224103097155692]

In [ ]:
import numpy as np 
np.array(metric).mean()

0.6761206550897432

In [ ]:
train['total_points'].std()

2.646924569263392

In [ ]:
model.fit(train.drop(columns='total_points'),train['total_points'])

pred = model.predict(test.drop(columns='total_points'))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010659 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 870
[LightGBM] [Info] Number of data points in the train set: 101488, number of used features: 20
[LightGBM] [Info] Start training from score 1.529245


In [ ]:
mean_squared_error(test['total_points'],pred,squared=False)

0.7236256667804261

In [ ]:
test['total_points'].std()

2.3552362999655774

In [ ]:
name

171                   Ben Davies
403                   Ben Davies
746                   Ben Davies
980                   Ben Davies
1330                  Ben Davies
                  ...           
24822            Dominic Ballard
25750            Dominic Ballard
26487    Yago de Santiago Alonso
26256          Samuel Amo-Ameyaw
26092             Mark Gillespie
Name: name, Length: 26505, dtype: object

In [ ]:
test['name'] = name

In [ ]:
test['name'] = list(name)

In [ ]:
test[test['name'] == 'Mo Salah']

,position,assists,bonus,bps,clean_sheets,fixture,goals_conceded,goals_scored,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,total_points,was_home,yellow_cards,GW,name
kickoff_time,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
test['predictions'] = pred

In [ ]:
test['year'] = test.index.dt.year

AttributeError: 'DatetimeIndex' object has no attribute 'dt'

In [ ]:
train['total_points'].max()

29

In [ ]:
test['date'] = test.index

In [ ]:
test['year'] = test['date'].dt.year

In [ ]:
test['name']

kickoff_time
2022-08-06 11:30:00+00:00                 Ben Davies
2022-08-06 14:00:00+00:00                 Ben Davies
2022-08-15 19:00:00+00:00                 Ben Davies
2022-08-14 15:30:00+00:00                 Ben Davies
2022-08-22 19:00:00+00:00                 Ben Davies
                                      ...           
2023-05-21 13:00:00+00:00            Dominic Ballard
2023-05-28 15:30:00+00:00            Dominic Ballard
2023-05-28 15:30:00+00:00    Yago de Santiago Alonso
2023-05-28 15:30:00+00:00          Samuel Amo-Ameyaw
2023-05-28 15:30:00+00:00             Mark Gillespie
Name: name, Length: 26505, dtype: object

In [ ]:
season_data = test.groupby(['year','name'])[['total_points','predictions']].agg('sum')

In [ ]:
season_data.sort_values('total_points',ascending=False)

total_points
year name                         
2023 Harry Kane                154
     Mohamed Salah             143
2022 Erling Haaland            142
2023 Erling Haaland            130
     Marcus Rashford           124
...                            ...
2022 Daniel Iversen              0
     Dane Scarlett               0
     Dale Taylor                 0
2023 Filip Marschall             0
2022 James Tomkins              -2

[1454 rows x 1 columns]

In [ ]:
season_data.sort_values('predictions',ascending=False)


total_points  predictions
year name                                             
2023 Harry Kane                       154   158.896045
2022 Erling Haaland                   142   145.059227
2023 Trent Alexander-Arnold           106   132.795783
2022 Kieran Trippier                  116   129.890343
2023 Bruno Borges Fernandes           119   129.166618
...                                   ...          ...
     Ishé Samuels-Smith                 0     0.007697
2022 Bashir Humphreys                   0    -0.000957
2023 Imari Samuels                      0    -0.002176
     Tayo Adaramola                     0    -0.005463
     Alfie Gilchrist                    0    -0.008134

[1454 rows x 2 columns]

In [ ]:
imp = pd.DataFrame({
    'name':model.feature_name_,
    'importance':model.feature_importance
    
})

AttributeError: 'LGBMRegressor' object has no attribute 'feature_importance'

In [ ]:
model.